In [27]:
import json
import twitter
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream, auth
import pandas as pd
import re
CONSUMER_KEY = 'JJNPMPuZubIrFIqDNARJRDPDb'
CONSUMER_SECRET = 'd6MlAqrJFcdyJac9aCQpBHPbwGs5eJB8zkTn5wA3tqBLHZgw0b'
OAUTH_TOKEN = '3229899732-piSMFy32Vi0VSyXJX8R9y2qrkr0piesoHXBdI3v'
OAUTH_TOKEN_SECRET = 'Jq9oTRMUjHRgA7NkJLLHIEyjtCRhYiFHdWkpBw28IBtHG'

#read emoji codes:
emoji_key = pd.read_excel('/Users/attiladobi/Documents/emoji_list.xlsx', encoding='utf-8', index_col=0, skiprows=1)
emj_codes=[code for code in emoji_key['Unicode'] if code!="Browser" if sum([c=="*" for c in code])==0]

def get_keys(tweet):
    max_index=0
    max_count=len(tweet['statuses'][max_index].keys())
    
    for ii in range(len(tweet['statuses'])):
        if max_count < len(tweet['statuses'][ii].keys()):
            max_index=ii
            
    return tweet['statuses'][max_index].keys()

def print_tweets_w_emj(T_DF,num=0):
    for item in T_DF['text'][T_DF['emjSum']>=num]:
        print(item)
        
def make_twitter_DF(tweet): #create PD dataframe
    columns=get_keys(tweet)
    for ii in range(len(tweet['statuses'])): 
        diff= list(set(columns)-set(tweet['statuses'][ii].keys()))
        for val in diff:
            tweet['statuses'][ii][val]=""
    return pd.DataFrame([tweet['statuses'][ii] for ii in range(len(tweet['statuses']))], columns=get_keys(tweet)) 

def Search_Word(q):
    # Initiate the connection to Twitter REST API
    twitter = Twitter(auth=OAuth( OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY,  CONSUMER_SECRET))       
    tweet= twitter.search.tweets(q=q,lang='en', result_type='recent',count=200)
    T_DF=make_twitter_DF(tweet)
    
    #long search function, more than 100
    #T_DF=long_search(twitter,q,300)
    
    T_DF['emjText']=[[(emcode, len(re.findall(emcode,T_DF['text'][ii]))) for emcode in emj_codes\
                      if (len(re.findall(emcode,T_DF['text'][ii])) > 0)] for ii in range(len(T_DF))]
    T_DF['emjSum']=[sum([item[1] for item in T_DF['emjText'][ii]]) for ii in range(len(T_DF))]
    T_DF['emjTypes']=[len(T_DF['emjText'][ii]) for ii in range(len(T_DF))]
    T_DF['searchWordSum']=[len(re.findall(q,T_DF['text'][ii].lower())) for ii in range(len(T_DF))]
    return T_DF

def Analyze_Freq(T_DF,emj_per_tweet=1,rep_emj_per_tweet=1, emj_types_per_tweet=1, nword=0):
    DFcut=T_DF[(T_DF['emjSum']>=emj_per_tweet) & (T_DF['searchWordSum']>=nword) \
               & (T_DF['emjTypes']>=emj_types_per_tweet)] 
    
    S=dict()
    for key, value in sum(DFcut['emjText'],[]):
        if value>=rep_emj_per_tweet:
            try:
                S[key] +=value
            except KeyError:
                S[key]=value
        
    return pd.DataFrame([(key,S[key]) for key in S],columns=('emoji','freq'))

In [43]:
twitter = Twitter(auth=OAuth( OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY,  CONSUMER_SECRET))       
tweet= twitter.search.tweets(q='hello',lang='en', result_type='recent',count=200)

In [49]:
type(tweet)

twitter.api.TwitterDictResponse

In [306]:
DFcut

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,...,retweeted,retweeted_status,source,text,truncated,user,emjText,emjSum,emjTypes,searchWordSum
4,None,None,Tue Mar 08 08:27:41 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 24...",0,False,None,707120591220940800,707120591220940800,None,...,False,"{u'contributors': None, u'truncated': False, u...","<a href=""https://about.twitter.com/products/tw...",RT @Crysd3: Shake my paw üêæüêæ\n\n#tunatuesday ...,False,"{u'follow_request_sent': False, u'has_extended...","[(üêæ, 2)]",2,1,1
5,None,None,Tue Mar 08 08:27:40 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 28...",0,False,None,707120588637265920,707120588637265920,140ron,...,False,,"<a href=""http://twitter.com/download/iphone"" r...",@140ron Haha...1 and 2...underway...may save t...,False,"{u'follow_request_sent': False, u'has_extended...","[(üòä, 1)]",1,1,1
6,None,None,Tue Mar 08 08:27:34 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 97...",0,False,None,707120564108795904,707120564108795904,None,...,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @syafiqahfarid: I'm sure Kwangsoo &amp; Hah...,False,"{u'follow_request_sent': False, u'has_extended...","[(üëç, 1)]",1,1,1
9,None,None,Tue Mar 08 08:27:25 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 18...",0,False,None,707120526603456512,707120526603456512,None,...,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @_reeeddd: @_JadeR1 I'm just gone shake it ...,False,"{u'follow_request_sent': False, u'has_extended...","[(üòÇ, 2)]",2,1,1
17,None,None,Tue Mar 08 08:26:58 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 33...",1,False,None,707120412749144064,707120412749144064,_JadeR1,...,False,,"<a href=""http://twitter.com/download/iphone"" r...",@_JadeR1 I'm just gone shake it off üòÇüòÇ,False,"{u'follow_request_sent': False, u'has_extended...","[(üòÇ, 2)]",2,1,1
26,None,None,Tue Mar 08 08:26:35 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,707120317374685184,707120317374685184,None,...,False,,"<a href=""http://twitter.com/download/android"" ...",hi aubrey! owen. *smile dala shake ng hands*\n...,False,"{u'follow_request_sent': False, u'has_extended...","[(üòÇ, 3)]",3,1,1
30,None,None,Tue Mar 08 08:26:28 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 97...",0,False,None,707120287678996481,707120287678996481,None,...,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @syafiqahfarid: I'm sure Kwangsoo &amp; Hah...,False,"{u'follow_request_sent': False, u'has_extended...","[(üëç, 1)]",1,1,1
31,None,None,Tue Mar 08 08:26:23 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,707120266716057600,707120266716057600,None,...,False,,"<a href=""http://instagram.com"" rel=""nofollow"">...",When your protein shakes running out üò¨ #prote...,False,"{u'follow_request_sent': False, u'has_extended...","[(üò¨, 1)]",1,1,2
36,None,None,Tue Mar 08 08:26:20 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 97...",0,False,None,707120253218590721,707120253218590721,None,...,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @syafiqahfarid: I'm sure Kwangsoo &amp; Hah...,False,"{u'follow_request_sent': False, u'has_extended...","[(üëç, 1)]",1,1,1
37,None,None,Tue Mar 08 08:26:18 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 97...",0,False,None,707120243752108032,707120243752108032,None,...,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @syafiqahfarid: I'm sure Kwangsoo &amp; Hah...,False,"{u'follow_request_sent': False, u'has_extended...","[(üëç, 1)]",1,1,1


In [305]:
DFcut=T_DF[(T_DF['emjSum']>=1) & (T_DF['searchWordSum']>=1) \
               & (T_DF['emjTypes']>=1)]
for key, value in sum(DFcut['emjText'],[]):
    print key,value

üêæ 2
üòä 1
üëç 1
üòÇ 2
üòÇ 2
üòÇ 3
üëç 1
üò¨ 1
üëç 1
üëç 1
üòè 1
ü§ë 1
üòã 1
üòç 1
üíï 1
üòã 1
üòç 1
üíï 1
üòÇ 1
üòÇ 1
üëÄ 1
üë∏ 1
üë∏üèº 1
üíÅ 1
üíÅüèº 1
üèº 2
üíÖ 1
üíó 1
üíÑ 1
üèª 4
üëé 4
üëéüèª 4
üíï 1
üêæ 2
üòÇ 1
üòÇ 1
üêæ 2
üé∂ 2
üòî 1
üò§ 1
üèæ 45
üñï 45
üñïüèæ 45
üòî 1
üò§ 1
üèæ 45
üñï 45
üñïüèæ 45
üòú 1
üêæ 2
üëç 1
üëç 1
üôÉ 1
üò£ 1
üë∏ 1
üë∏üèº 1
üíÅ 1
üíÅüèº 1
üèº 2
üíÖ 1
üíó 1
üíÑ 1
üôè 3
üíØ 1
üíØ 1
üòÇ 3
üëç 1
üëç 1
üåé 1
üëç 1
üò© 1
üôà 1
üëç 1
üíï 1
üòÄ 1
üòä 1
üòú 1
üëç 1
üôÅ 1
üëç 1
üòò 1
üòú 1
üëØ 1
üíÉ 1
üëç 1
üëç 1
üëç 1
üò∂ 1
üòã 1
üôã 1
üôãüèæ 1
üèæ 1
üí• 2
üçÄ 1
üî• 1
üëç 1
üëç 1
üòä 1
üí© 1
üòÇ 1
üëç 1
üòÑ 1
üôã 1
üôãüèæ 1
üèæ 1
üí• 2
üôå 1
üôåüèª 1
üèª 1
üçç 2
üòÅ 1
üôå 1
üë∏ 1
üë∏üèº 1
üíÅ 1
üíÅüèº 1
üèº 2
üíÖ 1
üíó 1
üíÑ 1
üòã 1
üòã 1
üòã 2
üç® 1
üòÇ 1
üòà 1
üíï 1
üò∑ 3
üòà 1
üíï 1
üëç 1
üéµ 1
üòã 1
üòç 1
üíï 1


In [325]:
emj_codes

[u'\U0001f600',
 u'\U0001f601',
 u'\U0001f602',
 u'\U0001f603',
 u'\U0001f604',
 u'\U0001f605',
 u'\U0001f606',
 u'\U0001f609',
 u'\U0001f60a',
 u'\U0001f60b',
 u'\U0001f60e',
 u'\U0001f60d',
 u'\U0001f618',
 u'\U0001f617',
 u'\U0001f619',
 u'\U0001f61a',
 u'\u263a\ufe0f',
 u'\U0001f642',
 u'\U0001f917',
 u'\U0001f607',
 u'\U0001f914',
 u'\U0001f610',
 u'\U0001f611',
 u'\U0001f636',
 u'\U0001f644',
 u'\U0001f60f',
 u'\U0001f623',
 u'\U0001f625',
 u'\U0001f62e',
 u'\U0001f910',
 u'\U0001f62f',
 u'\U0001f62a',
 u'\U0001f62b',
 u'\U0001f634',
 u'\U0001f60c',
 u'\U0001f913',
 u'\U0001f61b',
 u'\U0001f61c',
 u'\U0001f61d',
 u'\u2639',
 u'\U0001f641',
 u'\U0001f612',
 u'\U0001f613',
 u'\U0001f614',
 u'\U0001f615',
 u'\U0001f616',
 u'\U0001f643',
 u'\U0001f637',
 u'\U0001f912',
 u'\U0001f915',
 u'\U0001f911',
 u'\U0001f632',
 u'\U0001f61e',
 u'\U0001f61f',
 u'\U0001f624',
 u'\U0001f622',
 u'\U0001f62d',
 u'\U0001f626',
 u'\U0001f627',
 u'\U0001f628',
 u'\U0001f629',
 u'\U0001f62c',
 u'\U0001f

In [326]:
print(u"\U0001f609 \n hey")

üòâ 
 hey


In [25]:
def long_search(twitter,q,n):
    #tweets                          =   []
    MAX_ATTEMPTS                    =   10
    COUNT_OF_TWEETS_TO_BE_FETCHED   =   n 
    
    # STEP 1: Query Twitter
    # Query twitter for data. 
    tweets    = twitter.search.tweets(q=q, count='100', lang='en')
    results=tweets
    T_DF=make_twitter_DF(tweets)
    
    for i in range(0,MAX_ATTEMPTS):

        if(COUNT_OF_TWEETS_TO_BE_FETCHED < len(tweets['statuses'])):
            return T_DF
        try:
            next_results_url_params    = results['search_metadata']['next_results']
            next_max_id        = next_results_url_params.split('max_id=')[1].split('&')[0]
        except:
            return T_DF

        # After the first call we should have max_id from result of previous call. Pass it in query.
        results    = twitter.search.tweets(q=q,include_entities='true',max_id=next_max_id,count='100',lang='en')
        T_DF = pd.concat([T_DF,make_twitter_DF(results)],ignore_index=True)

<h1> Search a word </h1>

In [276]:
T_DF

,contributors,truncated,text,is_quote_status,in_reply_to_status_id,id,favorite_count,entities,quoted_status_id,retweeted,...,lang,created_at,quoted_status_id_str,in_reply_to_status_id_str,place,metadata,emjText,emjSum,emjTypes,searchWordSum
0,None,False,Why is Myrtle Sarosa wearing Taylor Swift's Sh...,False,NaN,707118162580340736,1,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,False,...,en,Tue Mar 08 08:18:02 +0000 2016,NaN,None,None,"{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1
1,None,False,shake it off,False,NaN,707118151532617728,0,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,False,...,en,Tue Mar 08 08:17:59 +0000 2016,NaN,None,None,"{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1
2,None,False,"My lips are drier than my timeline, and my hea...",False,NaN,707117569057103872,0,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,False,...,en,Tue Mar 08 08:15:40 +0000 2016,NaN,None,None,"{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1
3,None,False,@shslphantom do a Taylor swift and shake it of...,False,7.071135e+17,707116650533756928,1,"{u'symbols': [], u'user_mentions': [{u'id': 40...",NaN,False,...,en,Tue Mar 08 08:12:01 +0000 2016,NaN,707113488481689600,"{u'full_name': u'Los Angeles, CA', u'url': u'h...","{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1
4,None,False,Shake it off.,False,NaN,707115611076239360,0,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,False,...,en,Tue Mar 08 08:07:53 +0000 2016,NaN,None,None,"{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1
5,None,False,@BeastersKN 1 Roar 2 I really like you 3 Yeah ...,False,7.061150e+17,707114915572502530,0,"{u'symbols': [], u'user_mentions': [{u'id': 47...",NaN,False,...,en,Tue Mar 08 08:05:08 +0000 2016,NaN,706114977694765056,None,"{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1
6,None,False,I don't care how you feel about Taylor Swift. ...,False,NaN,707114803723157505,0,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,False,...,en,Tue Mar 08 08:04:41 +0000 2016,NaN,None,None,"{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1
7,None,False,Taylor Swift - Shake it off,False,NaN,707114435370995712,0,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,False,...,en,Tue Mar 08 08:03:13 +0000 2016,NaN,None,None,"{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1
8,None,False,Shake it off is op de radio https://t.co/WZBl4...,False,NaN,707114278797582336,0,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,False,...,en,Tue Mar 08 08:02:36 +0000 2016,NaN,None,None,"{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1
9,None,False,Dean likes Taylor Swift!! hahahahahah\nand he'...,False,NaN,707114215849336836,0,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,False,...,en,Tue Mar 08 08:02:21 +0000 2016,NaN,None,None,"{u'iso_language_code': u'en', u'result_type': ...",[],0,0,1


In [39]:
T_DF=Search_Word('taco')

In [40]:
ef=Analyze_Freq(T_DF,emj_per_tweet=1,rep_emj_per_tweet=1, emj_types_per_tweet=1, nword=0)
ef.sort('freq',ascending =0)

/Users/attiladobi/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from IPython.kernel.zmq import kernelapp as app


,emoji,freq
13,üôè,11
11,üåÆ,7
10,üòÇ,4
9,üôÑ,3
14,üò©,3
0,üòç,2
7,üèª,2
1,üê∑,1
2,üòë,1
3,üëç,1


In [41]:
print_tweets_w_emj(T_DF,1)

I had Taco Bell for lunch and I already want it again. lol üê∑
Let's go got an interview for Taco Bell tomorrow ü§ëüåÆüòã
my mom called while me and Colton were at the airport and said "you know my plane doesn't come in until tomorrow right" üòÇüòÇ
@Stevi_Rae I seriously just got taco bellüò© later!!!???
I ordered chips and queso from Taco Cabana and they forgot the chips but gave me the queso üòë
Ugh Taco Bell sounds wonderful üò©
RT @MIZZOU_DU: üåÆüåÆüåÆüåÆüåÆüåÆ
TOMORROW!
Saved by the Bell 
Unlimited Taco Bell for $5!
March 17 : 10pm-2am
Contact Gust‚Ä¶ https://t.co/Fxox37nXgL
@duhhnielaa @Dodgers their snaps yesterday had me hurt I wasn't invited to taco Tuesday üôÑüôÑüôÑ
RT @dntgve: Can we make it Taco wensday too @MexicanFood üôèüôèüôèüôèüôèüôèüôèüôèüôèüôèüôè https://t.co/Z8FiWxKTlr
that was the most fun I've ever had cleaning my car üòÇ
Just saw the cutest guy working at Taco Bell omfg üòçüòçüò©
I'd make the worst personal trainer ever, "Go, be f

In [292]:
str(ef['emoji'])

'0       \xf0\x9f\x96\x95\n1     \xf0\x9f\x92\x83\xf0\x9f\x8f\xbe\n2       \xf0\x9f\x98\xb7\n3       \xf0\x9f\x98\x91\n4       \xf0\x9f\x8f\xbe\n5       \xf0\x9f\x8f\xbb\n6       \xf0\x9f\x98\x85\n7     \xf0\x9f\x96\x95\xf0\x9f\x8f\xbe\n8       \xf0\x9f\x92\x83\n9       \xf0\x9f\x98\x82\n10    \xf0\x9f\x91\x8e\xf0\x9f\x8f\xbb\n11      \xf0\x9f\x91\x8e\n12      \xf0\x9f\x98\xa9\n13      \xf0\x9f\x99\x8f\nName: emoji, dtype: object'